In [1]:
!pip install -q pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.config('spark.ui.port', '4050').getOrCreate()
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType,DateType

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [4]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

rm: cannot remove '/root/.kaggle': No such file or directory


In [5]:
!kaggle datasets download -d evanschreiner/netflix-movie-ratings

100% 684M/685M [00:23<00:00, 32.9MB/s]
100% 685M/685M [00:23<00:00, 30.8MB/s]


In [9]:
!ls
!pwd

movies.csv  netflix-movie-ratings.zip  Netflix_User_Ratings.csv  sample_data
/content


In [7]:
!unzip netflix-movie-ratings.zip

Archive:  netflix-movie-ratings.zip
  inflating: Netflix_User_Ratings.csv  
  inflating: movies.csv              


In [19]:
movies_schema = StructType([StructField('movie_id',IntegerType(),nullable=False),
                            StructField('release_year',IntegerType(),nullable=False),
                            StructField('movie_title',StringType(),nullable=False),
    
])

In [20]:
moviesDF = spark.read.csv(path='/content/movies.csv',header=True,schema=movies_schema)

In [21]:
moviesDF.show(5)

+--------+------------+--------------------+
|movie_id|release_year|         movie_title|
+--------+------------+--------------------+
|       1|        2003|     Dinosaur Planet|
|       2|        2004|Isle of Man TT 20...|
|       3|        1997|           Character|
|       4|        1994|Paula Abdul's Get...|
|       5|        2004|The Rise and Fall...|
+--------+------------+--------------------+
only showing top 5 rows



In [22]:
user_rating_schema = StructType([StructField('cust_id',IntegerType(),nullable=False),
                                 StructField('rating',IntegerType(),nullable=False),
                                 StructField('date',DateType(),nullable=False),
                                 StructField('movie_id',IntegerType(),nullable=False),
    
])

In [23]:
netflixUserRatingsDF = spark.read.csv(path='/content/Netflix_User_Ratings.csv',header=True,schema=user_rating_schema)

In [24]:
netflixUserRatingsDF.show(5)

+-------+------+----------+--------+
|cust_id|rating|      date|movie_id|
+-------+------+----------+--------+
|1488844|     3|2005-09-06|       1|
| 822109|     5|2005-05-13|       1|
| 885013|     4|2005-10-19|       1|
|  30878|     4|2005-12-26|       1|
| 823519|     3|2004-05-03|       1|
+-------+------+----------+--------+
only showing top 5 rows



In [25]:
moviesDF.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- movie_title: string (nullable = true)



In [26]:
netflixUserRatingsDF.printSchema()

root
 |-- cust_id: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- movie_id: integer (nullable = true)



In [35]:
movie_title_col = moviesDF.select('movie_title',(moviesDF['release_year']+10).alias('new_release_year'))

In [39]:
movies_2003_df = moviesDF.filter(moviesDF['release_year']==2003).show(10)

+--------+------------+--------------------+
|movie_id|release_year|         movie_title|
+--------+------------+--------------------+
|       1|        2003|     Dinosaur Planet|
|      13|        2003|Lord of the Rings...|
|      30|        2003|Something's Gotta...|
|      34|        2003|Ashtanga Yoga: Be...|
|      38|        2003|  Daydream Obsession|
|      49|        2003|Devo: The Complet...|
|      53|        2003|      The Bonesetter|
|      59|        2003|Sex and the Beauties|
|      69|        2003|WWE: Armageddon 2003|
|     111|        2003| Duplex (Widescreen)|
+--------+------------+--------------------+
only showing top 10 rows



In [40]:
groupedDF = netflixUserRatingsDF.groupBy('movie_id').count()

In [41]:
groupedDF.show(5)

+--------+-----+
|movie_id|count|
+--------+-----+
|     148|22194|
|     463| 2286|
|     471| 4500|
|     496|  235|
|     833| 8378|
+--------+-----+
only showing top 5 rows



In [46]:
from pyspark.sql.functions import col, max

TypeError: ignored

In [49]:
groupedDF.select(max(col("count"))).show()

+----------+
|max(count)|
+----------+
|    232944|
+----------+



In [56]:
result = groupedDF.filter(groupedDF['count']==232944).show()

+--------+------+
|movie_id| count|
+--------+------+
|    5317|232944|
+--------+------+



In [57]:
moviesDF.filter(moviesDF['movie_id']==5317).show(5)

+--------+------------+-----------------+
|movie_id|release_year|      movie_title|
+--------+------------+-----------------+
|    5317|        2000|Miss Congeniality|
+--------+------------+-----------------+

